In [1]:
from __future__ import division

from models import *
from utils.logger import *
from utils.utils import *
from utils.datasets import *
from utils.parse_config import *
from test import evaluate

import os
import sys
import time
import shutil
import datetime
import argparse
import cv2
from PIL import Image
from terminaltables import AsciiTable

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
from torchvision import transforms

import torch.optim as optim
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.ticker import NullLocator

In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--epochs", type=int, default=100, help="number of epochs")
    parser.add_argument("--batch_size", type=int, default=8, help="size of each image batch")
    parser.add_argument("--gradient_accumulations", type=int, default=2, help="number of gradient accums before step")
    parser.add_argument("--model_def", type=str, default="config/yolov3-kitti.cfg", help="path to model definition file")
    parser.add_argument("--data_config", type=str, default="config/kitti-ssl.data", help="path to data config file")
    parser.add_argument("--pretrained_weights", type=str, help="if specified starts from checkpoint model")
    parser.add_argument("--nms_thres", type=float, default=0.4, help="iou thresshold for non-maximum suppression")
    parser.add_argument("--conf_thres", type=float, default=0.9, help="object confidence threshold")
    parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
    parser.add_argument("--img_size", type=int, default=416, help="size of each image dimension")
    parser.add_argument("--checkpoint_interval", type=int, default=1, help="interval between saving model weights")
    parser.add_argument("--evaluation_interval", type=int, default=1, help="interval evaluations on validation set")
    parser.add_argument("--compute_map", default=False, help="if True computes mAP every tenth batch")
    parser.add_argument("--multiscale_training", default=True, help="allow for multi-scale training")
    opt = parser.parse_args()
    print(opt)
    
    logger = Logger("logs")
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
    os.makedirs("output_train", exist_ok=True)
    os.makedirs("checkpoint_train", exist_ok=True)
    
    pseudo_rej = []
    pseudo_train = []

In [3]:
    # Get data configuration
    data_config = 'config/yolov3-kitti.cfg'
    train_path = 'data/pseudo/pseudo_train_rest.txt'
    valid_path = 'data/valid.txt'
    predict_path = 'data/test.txt'
    class_names = load_classes('data/kitti.names')
    weights = torch.load('checkpoint_weights/yolov3_ckpt_10%.pth')
    print(weights)
    

OrderedDict([('module_list.0.conv_0.weight', tensor([[[[ 3.6996e-02,  4.5037e-03,  1.0608e-02],
          [ 3.6211e-02, -1.0329e-02,  3.8759e-02],
          [-2.4291e-02, -7.3165e-02, -1.8108e-02]],

         [[-2.7783e-02, -6.7625e-02, -3.8481e-02],
          [-3.0037e-02, -6.8715e-02, -6.2428e-03],
          [-4.6438e-02, -8.5287e-02, -2.4306e-02]],

         [[ 5.4486e-02,  4.0257e-02,  3.1781e-02],
          [ 3.1174e-02, -1.3436e-03,  4.5569e-02],
          [-4.5752e-02, -6.5742e-02, -7.7133e-03]]],


        [[[-4.3815e-02, -2.6904e-02, -8.4694e-03],
          [-3.0887e-02, -2.1751e-02, -1.9598e-02],
          [-1.3751e-02, -1.4129e-02, -5.0179e-02]],

         [[-7.0319e-02, -4.2799e-02,  3.0551e-02],
          [-3.6146e-02, -1.9152e-02,  3.0432e-02],
          [ 2.6630e-02,  2.5597e-02,  2.4225e-02]],

         [[-4.6978e-02, -3.0370e-02, -1.0809e-02],
          [-2.2345e-02, -7.2073e-03,  6.9121e-03],
          [ 1.3635e-02,  1.2073e-02, -6.4980e-03]]],


        [[[-4.0548e-0

In [4]:
    for epoch in range(opt.epochs):
    
        # Set up model
        model = Darknet(opt.model_def, img_size=416).to(device)

        # Load checkpoint weights
        model.load_state_dict(torch.load(weights))

        model.eval()  # Set in evaluation mode

        dataloader = DataLoader(
            ListImages(predict_path, img_size=opt.img_size),
            batch_size=1,
            shuffle=False,
            num_workers=opt.n_cpu,
        )

        Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

        imgs = []  # Stores image paths
        img_detections = []  # Stores detections for each image index    
        print("\nPerforming object detection:")
        prev_time = time.time()
        for batch_i, (img_paths, input_imgs) in enumerate(dataloader):
            # Configure input
            input_imgs = Variable(input_imgs.type(Tensor))

            # Get detections
            with torch.no_grad():
                detections = model(input_imgs)
                detections = non_max_suppression(detections, opt.conf_thres, opt.nms_thres)

            # Log progress
            current_time = time.time()
            inference_time = datetime.timedelta(seconds=current_time - prev_time)
            prev_time = current_time
            print("\t+ Batch %d, Inference Time: %s" % (batch_i, inference_time))

            # Save image and detections
            imgs.extend(img_paths)
            img_detections.extend(detections)

NameError: name 'opt' is not defined

In [ ]:
        print("\nSaving labels:")
        # Iterate through images and save plot of detections
        for img_i, (path, detections) in enumerate(zip(imgs, img_detections)):

            # Create plot
            img = cv2.imread(path)
            img_height, img_width = img.shape[0],img.shape[1]
            label_to_write = []

            # Draw bounding boxes and labels of detections
            if detections is not None:
                # Rescale boxes to original image
                #print('detections',detections)
                detections = rescale_boxes(detections, opt.img_size, img.shape[:2])
                unique_labels = detections[:, -1].cpu().unique()
                n_cls_preds = len(unique_labels)

                for x1, y1, x2, y2, conf, cls_conf, cls_pred in detections:

                    vehicle_class = [2,5,7]

                    if int(cls_pred) in vehicle_class:
                        cls_pred = 1
                    else:
                        cls_pred = 0

                    print("\t+ Label: %s, Conf: %.5f" % (class_names[int(cls_pred)], cls_conf.item()))

                    #print("Bounding Box:", class_str,x1,y1,w,h)
                    intx1 = int(x1)
                    inty1 = int(y1)
                    intx2 = int(x2)
                    inty2 = int(y2)

                    bbox_center_x = float( (x1 + (x2 - x1) / 2.0) / img_width)
                    bbox_center_y = float( (y1 + (y2 - y1) / 2.0) / img_height)
                    bbox_width = float((x2 - x1) / img_width)
                    bbox_height = float((y2 - y1) / img_height)

                    line_to_write = str(str(cls_pred)) + ' ' + str(bbox_center_x)+ ' ' + str(bbox_center_y) + ' ' +str(bbox_width)+ ' ' + str(bbox_height) +'\n'
                    label_to_write.extend(line_to_write)

            # Save generated detections as pseudo labels
            if len(label_to_write) > 0:
                filename = path.split("/")[-1].split(".")[0]
                label_data = open(f"data/pseudo/labels/{filename}.txt",'w')
                for line in label_to_write:
                    label_data.write(line)
                label_data.close()
                pseudo_train.append(str('/home/aditya/yolov3/final/data/pseudo/images/'+filename+'.png'))
            else:
                pseudo_rej.append(str(str(opt.image_folder)+"/"+filename+".png"))


        unique_src_path = list(dict.fromkeys(pseudo_rej))

        with open('data/pseudo/pseudo_train_rejected_list.txt', 'w') as f:
            for src in unique_src_path:
                f.write("%s\n" % src)
        with open('data/pseudo/pseudo_train_rest.txt', 'w') as f:
            for path in pseudo_train:
                f.write("%s\n" % path)

        print("Labels Generated.")


In [ ]:
        #Begin training

        # Initiate model
        model = Darknet(opt.model_def).to(device)
        model.apply(weights_init_normal)

        #Load weights
        model.load_state_dict(weights)

        # Get dataloader
        dataset = ListDataset(train_path, augment=True, multiscale=opt.multiscale_training)
        dataloader = torch.utils.data.DataLoader(
            dataset,
            batch_size=opt.batch_size,
            shuffle=True,
            num_workers=opt.n_cpu,
            pin_memory=True,
            collate_fn=dataset.collate_fn,
        )

        optimizer = torch.optim.Adam(model.parameters())

        metrics = [
            "grid_size",
            "loss",
            "x",
            "y",
            "w",
            "h",
            "conf",
            "cls",
            "cls_acc",
            "recall50",
            "recall75",
            "precision",
            "conf_obj",
            "conf_noobj",
        ]

In [ ]:
        #for epoch in range(opt.epochs):
        model.train()
        start_time = time.time()
        for batch_i, (_, imgs, targets) in enumerate(dataloader):
            batches_done = len(dataloader) * epoch + batch_i

            imgs = Variable(imgs.to(device))
            targets = Variable(targets.to(device), requires_grad=False)

            loss, outputs = model(imgs, targets)
            loss.backward()

            if batches_done % opt.gradient_accumulations:
                # Accumulates gradient before each step
                optimizer.step()
                optimizer.zero_grad()

            # ----------------
            #   Log progress
            # ----------------

            log_str = "\n---- [Epoch %d/%d, Batch %d/%d] ----\n" % (epoch, opt.epochs, batch_i, len(dataloader))

            metric_table = [["Metrics", *[f"YOLO Layer {i}" for i in range(len(model.yolo_layers))]]]

            # Log metrics at each YOLO layer
            for i, metric in enumerate(metrics):
                formats = {m: "%.6f" for m in metrics}
                formats["grid_size"] = "%2d"
                formats["cls_acc"] = "%.2f%%"
                row_metrics = [formats[metric] % yolo.metrics.get(metric, 0) for yolo in model.yolo_layers]
                metric_table += [[metric, *row_metrics]]

                # Tensorboard logging
                tensorboard_log = []
                for j, yolo in enumerate(model.yolo_layers):
                    for name, metric in yolo.metrics.items():
                        if name != "grid_size":
                            tensorboard_log += [(f"{name}_{j+1}", metric)]
                tensorboard_log += [("loss", loss.item())]
                logger.list_of_scalars_summary(tensorboard_log, batches_done)

            log_str += AsciiTable(metric_table).table
            log_str += f"\nTotal loss {loss.item()}"

            # Determine approximate time left for epoch
            epoch_batches_left = len(dataloader) - (batch_i + 1)
            time_left = datetime.timedelta(seconds=epoch_batches_left * (time.time() - start_time) / (batch_i + 1))
            log_str += f"\n---- ETA {time_left}"

            print(log_str)

            model.seen += imgs.size(0)

        if epoch % opt.evaluation_interval == 0:
            print("\n---- Evaluating Model ----")
            # Evaluate the model on the validation set
            precision, recall, AP, f1, ap_class = evaluate(
                model,
                path=valid_path,
                iou_thres=0.5,
                conf_thres=0.5,
                nms_thres=0.5,
                img_size=opt.img_size,
                batch_size=8,
            )
            evaluation_metrics = [
                ("val_precision", precision.mean()),
                ("val_recall", recall.mean()),
                ("val_mAP", AP.mean()),
                ("val_f1", f1.mean()),
            ]
            logger.list_of_scalars_summary(evaluation_metrics, epoch)

            # Print class APs and mAP
            ap_table = [["Index", "Class name", "AP"]]
            for i, c in enumerate(ap_class):
                ap_table += [[c, class_names[c], "%.5f" % AP[i]]]
            print(AsciiTable(ap_table).table)
            print(f"---- mAP {AP.mean()}")

        if epoch % opt.checkpoint_interval == 0:
            torch.save(model.state_dict(), f"checkpoint_train/yolov3_ckpt_%d.pth" % epoch)
            weights = model.state_dict()
